In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score



In [21]:
df = pd.read_csv("../data/processed/clinicaltrials_features_ml.csv")

df.shape
df.head()


,NCTId,PrimaryConditionMesh,DurationPrimaryDays,DurationCompletionDays,StartYear,StartMonth,StudyFirstSubmitYear,LastUpdateYear,PrimaryInterventionMesh,EnrollmentCount,...,PrimaryCountry_reduced_China,PrimaryCountry_reduced_Egypt,PrimaryCountry_reduced_France,PrimaryCountry_reduced_Germany,PrimaryCountry_reduced_Japan,PrimaryCountry_reduced_Other,PrimaryCountry_reduced_South Korea,PrimaryCountry_reduced_United Kingdom,PrimaryCountry_reduced_United States,PrimaryCountry_reduced_Unknown
0,NCT01315821,"Enterocolitis, Necrotizing",303.0,303.0,2011,2,2011,2011,Unknown,220.0,...,False,False,False,False,False,True,False,False,False,False
1,NCT04551521,Neoplasm Metastasis,1174.0,1174.0,2021,10,2020,2025,Vemurafenib,72.0,...,False,False,False,True,False,False,False,False,False,False
2,NCT04086121,"Dermatitis, Atopic",582.0,883.0,2019,9,2019,2025,spesolimab,14.0,...,False,False,False,False,False,False,False,False,True,False
3,NCT01181921,Alzheimer Disease,31.0,31.0,2011,5,2010,2014,Galantamine,1.0,...,False,False,False,False,False,True,False,False,False,False
4,NCT05435014,"Carcinoma, Hepatocellular",1386.0,1386.0,2022,9,2021,2024,Ethiodized Oil,90.0,...,False,False,False,False,False,True,False,False,False,False


In [22]:
# Crear columna target como en 03
df["target"] = np.nan

df.loc[df["OverallStatus_COMPLETED"] == 1, "target"] = 1
df.loc[
    df[["OverallStatus_TERMINATED",
        "OverallStatus_WITHDRAWN",
        "OverallStatus_SUSPENDED"]].sum(axis=1) > 0,
    "target"
] = 0

# Nos quedamos solo con filas con target definido
df = df.dropna(subset=["target"]).copy()
df["target"] = df["target"].astype(int)

df["target"].value_counts(normalize=True)


target
1    0.813371
0    0.186629
Name: proportion, dtype: float64

In [23]:
X = df.drop(columns=["target", "NCTId","PrimaryConditionMesh","PrimaryInterventionMesh","OverallStatus_TERMINATED",
                     "OverallStatus_COMPLETED","OverallStatus_WITHDRAWN","OverallStatus_SUSPENDED"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [24]:
X_train

,DurationPrimaryDays,DurationCompletionDays,StartYear,StartMonth,StudyFirstSubmitYear,LastUpdateYear,EnrollmentCount,Sex_ALL,Sex_FEMALE,Sex_MALE,...,PrimaryCountry_reduced_China,PrimaryCountry_reduced_Egypt,PrimaryCountry_reduced_France,PrimaryCountry_reduced_Germany,PrimaryCountry_reduced_Japan,PrimaryCountry_reduced_Other,PrimaryCountry_reduced_South Korea,PrimaryCountry_reduced_United Kingdom,PrimaryCountry_reduced_United States,PrimaryCountry_reduced_Unknown
146915,184.0,214.0,2011,5,2010,2017,27.0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
116791,876.0,975.0,2019,12,2019,2025,23.0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
130619,1302.0,1302.0,2018,5,2018,2022,7.0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
180403,335.0,1126.0,2004,7,2007,2008,20.0,0,1,0,...,False,False,False,False,False,False,False,False,False,True
62527,2598.0,2850.0,2009,2,2009,2018,22.0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120519,303.0,760.0,2004,9,2005,2014,601.0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
143831,1926.0,1926.0,2017,6,2017,2022,28.0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
97348,3714.0,3714.0,2002,5,2013,2014,330.0,0,1,0,...,False,False,False,False,False,True,False,False,False,False
57598,671.0,671.0,2014,4,2014,2016,48.0,1,0,0,...,False,False,False,False,False,True,False,False,False,False


In [25]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

print("AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


AUC: 0.8531008499327694
              precision    recall  f1-score   support

           0       0.85      0.46      0.59      5404
           1       0.89      0.98      0.93     23552

    accuracy                           0.88     28956
   macro avg       0.87      0.72      0.76     28956
weighted avg       0.88      0.88      0.87     28956



In [ ]:
param_dist = {
    'n_estimators': [100, 200, 400,600],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5,10],
    'min_samples_leaf': [1, 2,3],
    'bootstrap': [True, False]
}

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring="roc_auc"
)

rf_random.fit(X_train, y_train)
rf_random.best_params_


Fitting 2 folds for each of 20 candidates, totalling 40 fits


In [ ]:
hgb = HistGradientBoostingClassifier(random_state=42)
hgb.fit(X_train, y_train)

y_pred = hgb.predict(X_test)
y_proba = hgb.predict_proba(X_test)[:, 1]

print("AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


In [ ]:
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:, 1]

print("AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist_xgb = {
    "n_estimators": [200, 400,600,800,1000],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.5, 0.7, 1],
    "colsample_bytree": [0.5, 0.7, 1],
    "gamma": [0, 1, 5]
}

xgb_random = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist_xgb,
    n_iter=20,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring="roc_auc"
)

xgb_random.fit(X_train, y_train)
xgb_random.best_params_


In [ ]:
models = {
    "Random Forest": rf_random.best_estimator_,
    "HistGradientBoosting": hgb,
    "XGBoost": xgb_random.best_estimator_
}

results = {}

for name, model in models.items():
    proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, proba)
    results[name] = auc

results

#LEGALES 

In [ ]:
rf_importances = pd.Series(
    rf_random.best_estimator_.feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)

rf_importances.head(20)


In [ ]:
xgb_random.best_estimator_.get_booster().get_score(importance_type="gain")
